In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/sign-language-mnist/sign_mnist_train.csv')
dtest = pd.read_csv('/kaggle/input/sign-language-mnist/sign_mnist_test.csv')
df.sample(frac=1)
dtest.sample(frac=1)

In [ ]:
labeltoletter = {i:chr(i + ord('A')) for i in range(26)}


In [ ]:
df.describe()

In [ ]:
print(df.groupby('label').size())
df.groupby('label').size().mean()

## letter J(9) and Z(25) are missing as mentioned in the description 
# Average is about 11.44
## letter E has little less data


In [ ]:
df.isna().sum().sum() 
# hence no null value

In [ ]:
y_train,y_test = df['label'],dtest['label']

In [ ]:
X_train,X_test = df.drop(['label'],axis = 1),dtest.drop(['label'],axis = 1)

In [ ]:
X_train.iloc[0]

In [ ]:
import matplotlib.pyplot as plt
fig_name = 0
plt.imshow(np.array(X_train.iloc[fig_name]).reshape(28,28))
plt.title(f'label is {y_train.loc[fig_name]} representing {labeltoletter[y_train.loc[fig_name]]}')
plt.colorbar()

In [ ]:
# We need to rescale the input
X_train,X_test = X_train/255.,X_test/255.
X_train.describe()

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    
    plt.imshow(np.array(X_train.iloc[i]).reshape(28,28), cmap=plt.cm.binary)
    plt.xlabel(labeltoletter[y_train.loc[i]])
#     plt.colorbar()
plt.show()
    

## 

In [ ]:
import tensorflow as tf

print(tf.__version__)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128,input_shape=(1,784),activation = 'relu'),
    tf.keras.layers.Dense(64,activation = 'relu'),
    tf.keras.layers.Dense(32,activation = 'relu'),
    tf.keras.layers.Dense(26,activation = 'softmax')
])
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,y_train, epochs=20,validation_data=(X_test,y_test))

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['accuracy'],label = 'Train accuracy')
plt.plot(history.history['val_accuracy'],label = 'Test Accuracy')
plt.xticks([i for i in range(1,21)])
plt.legend()
plt.show()

# CONVOLUTION

In [ ]:
model_conv = tf.keras.Sequential([
    tf.keras.layers.Reshape((28,28,1),input_shape=(784,1)),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32,activation = 'relu'),
    tf.keras.layers.Dense(26,activation = 'sigmoid')
])
model_conv.summary()

In [ ]:
model_conv.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
history_conv = model_conv.fit(X_train,y_train, epochs=15,validation_data=(X_test,y_test))


In [ ]:
# plt.plot(history.history['accuracy'],label = 'Train accuracy')
# plt.plot(history.history['val_accuracy'],label = 'Test Accuracy')
plt.plot(history_conv.history['accuracy'],label = 'Train-Conv accuracy')
plt.plot(history_conv.history['val_accuracy'],label = 'Test-conv Accuracy')
plt.xticks([i for i in range(1,21,2)])
plt.xlim(0,21)
plt.legend()
plt.show()

In [ ]:
y_pred = np.argmax(model_conv.predict(X_test),axis = 1)
# print(len(Y_pred),len(y_test))


In [ ]:
for i in range(len(y_pred)):
    if y_pred[i] != y_test[i]:
#         print(i)
        plt.imshow(np.array(X_test.iloc[i]).reshape(28,28))
        plt.title(f'Predicted as {y_pred[i]} but was {y_test[i]}')
        
        

In [ ]:
y_pred[42]

In [ ]:
y_test

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
#   tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
  tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)
])

In [ ]:
model_conv_aug = tf.keras.Sequential([
    tf.keras.layers.Reshape((28,28,1),input_shape=(784,1)),
#     data_augmentation,
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.Dropout(0.7),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.Dropout(0.7),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32,activation = 'relu'),
    tf.keras.layers.Dense(26,activation = 'sigmoid')
])
model_conv_aug.summary()

In [ ]:
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience = 2, 
                                            verbose=1,factor=0.25, 
                                            min_lr=0.0001)
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.998):
            print("\nReached 99.8% accuracy so cancelling training!")
            self.model.stop_training = True


In [ ]:
model_conv_aug.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
history_conv_aug = model_conv_aug.fit(X_train,y_train, epochs=25,validation_data=(X_test,y_test),callbacks = [learning_rate_reduction,myCallback()])


In [ ]:
plt.plot(history_conv_aug.history['accuracy'],label = 'Train-Conv-aug accuracy')
# plt.plot(history_conv.history['val_accuracy'],label = 'Test-conv Accuracy')
plt.plot(history_conv_aug.history['val_accuracy'],label = 'Test-conv-aug Accuracy')
plt.xticks([i for i in range(1,21,2)])
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
# Predict the label of the test_images
pred = model_conv_aug.predict(X_test)
pred = np.argmax(pred,axis=1)

# Get the accuracy score
acc = accuracy_score(y_test,pred)

# Display the results
print(f'## {acc*100:.2f}% accuracy on the test set')

In [ ]:
y_test_letters = [labeltoletter[x] for x in y_test]
pred_letters = [labeltoletter[x] for x in pred]

print(classification_report(y_test_letters, pred_letters))

In [ ]:
import seaborn as sns
cf_matrix = confusion_matrix(y_test_letters, pred_letters, normalize='true')
plt.figure(figsize = (20,15))
sns.heatmap(cf_matrix, annot=True, xticklabels = sorted(set(y_test_letters)), yticklabels = sorted(set(y_test_letters)),cbar=False)
plt.title('Normalized Confusion Matrix\n', fontsize = 23)
plt.xlabel("Predicted Classes",fontsize=15)
plt.ylabel("True Classes",fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15,rotation=0)
plt.show()